<a href="https://colab.research.google.com/github/CUknot/NLP/blob/main/HW6_text_classification_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HOMEWORK 6: TEXT CLASSIFICATION
In this homework, you will create models to classify texts from TRUE call-center. There are two classification tasks:
1. Action Classification: Identify which action the customer would like to take (e.g. enquire, report, cancle)
2. Object Classification: Identify which object the customer is referring to (e.g. payment, truemoney, internet, roaming)

We will focus only on the Object Classification task for this homework.

In this homework, you are asked compare different text classification models in terms of accuracy and inference time.

You will need to build 3 different models.

1. A model based on tf-idf
2. A model based on MUSE
3. A model based on wangchanBERTa

**You will be ask to submit 3 different files (.pdf from .ipynb) that does the 3 different models. Finally, answer the accuracy and runtime numbers in MCV.**

This homework is quite free form, and your answer may vary. We hope that the processing during the course of this assignment will make you think more about the design choices in text classification.

In [ ]:
!wget --no-check-certificate https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv

--2025-02-14 16:07:25--  https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/8h8hvsw9uj6o0524lfe4i/clean-phone-data-for-students.csv?rlkey=lwv5xbf16jerehnv3lfgq5ue6 [following]
--2025-02-14 16:07:26--  https://www.dropbox.com/scl/fi/8h8hvsw9uj6o0524lfe4i/clean-phone-data-for-students.csv?rlkey=lwv5xbf16jerehnv3lfgq5ue6
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc08437ddcfd9813591a4edd0ea0.dl.dropboxusercontent.com/cd/0/inline/CkFPyyl-HjeFee1zj3-0jC6_v7aOHN3jBNlrswGFk7EgEvFuetzd06EHfNIbQ-gjeDhilKAYukiCO0yRHmeBiR1m0hULiInjoKmgm-WZgxfXGKxd6-uyNYML9eHv1g0wt0k/file# [following]
--2025-02-14 16:07:26--  https://uc08437ddcfd9813591a4ed

In [ ]:
!pip install pythainlp

## Import Libs

In [ ]:
%matplotlib inline
import pandas
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from torch.utils.data import Dataset
from IPython.display import display
from collections import defaultdict
from sklearn.metrics import accuracy_score

## Loading data
First, we load the data from disk into a Dataframe.

A Dataframe is essentially a table, or 2D-array/Matrix with a name for each column.

In [ ]:
data_df = pd.read_csv('clean-phone-data-for-students.csv')

Let's preview the data.

In [ ]:
# Show the top 5 rows
display(data_df.head())
# Summarize the data
data_df.describe()

,Sentence Utterance,Action,Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues


,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


## Data cleaning

We call the DataFrame.describe() again.
Notice that there are 33 unique labels/classes for object and 10 unique labels for action that the model will try to predict.
But there are unwanted duplications e.g. Idd,idd,lotalty_card,Lotalty_card

Also note that, there are 13389 unqiue sentence utterances from 16175 utterances. You have to clean that too!

## #TODO 0.1:
You will have to remove unwanted label duplications as well as duplications in text inputs.
Also, you will have to trim out unwanted whitespaces from the text inputs.
This shouldn't be too hard, as you have already seen it in the demo.



In [ ]:
display(data_df.describe())
display(data_df.Object.unique())
display(data_df.Action.unique())

,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nonTrueMove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd',
       'TrueMoney', 'garbage', 'Payment', 'IDD', 'ringtone', 'Idd',
       'rate', 'loyalty_card', 'contact', 'officer', 'Balance', 'Service',
       'Loyalty_card'], dtype=object)

array(['enquire', 'report', 'cancel', 'Enquire', 'buy', 'activate',
       'request', 'Report', 'garbage', 'change'], dtype=object)

In [ ]:
# TODO.1: Data cleaning
data_df = data_df.drop(columns=["Action"])
data_df.columns = ['input', 'label']
data_df.head()

,input,label
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,phone_issues


In [ ]:
data_df['input']=data_df['input'].str.strip().str.lower()
data_df['label']=data_df['label'].str.strip().str.lower()

In [ ]:
data_df = data_df.drop_duplicates("input", keep='first')

In [ ]:
data_df.shape

(13362, 2)

In [ ]:
data_df = data_df.reset_index(drop=True)

In [ ]:
# Create label map
label_2_idx = {l:i for i,l in enumerate(data_df.label.unique())}
idx_2_label = {i:l for l,i in label_2_idx.items()}

In [ ]:
data_df['label'] = data_df['label'].map(label_2_idx)

In [ ]:
data_df.head()

,input,label
0,<phone_number_removed> ผมไปจ่ายเงินที่ counter...,0
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,1
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้ ค่ะ,2
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อง...,3
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโท...,4


Split data into train, valdation, and test sets (normally the ratio will be 80:10:10 , respectively). We recommend to use train_test_spilt from scikit-learn to split the data into train, validation, test set.

In addition, it should split the data that distribution of the labels in train, validation, test set are similar. There is **stratify** option to handle this issue.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Make sure the same data splitting is used for all models.

In [ ]:
data_df['label'].value_counts()

,count
label,
5,2105
1,1790
3,1786
7,1478
11,1142
2,730
0,641
4,581
9,540


In [ ]:
from sklearn.model_selection import train_test_split

# remove class `contact` as it has only 4 sample (cannot do train 80 val 10 test 10)
data_df = data_df[data_df.label != 24]

X = data_df['input']
y = data_df['label']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [ ]:
# prompt: print X_train, y_train, X_val, y_val, X_test, y_test size with text lable in printf

print(f"X_train size: {X_train.shape[0]}, y_train size: {y_train.shape[0]}")
print(f"X_val size: {X_val.shape[0]}, y_val size: {y_val.shape[0]}")
print(f"X_test size: {X_test.shape[0]}, y_test size: {y_test.shape[0]}")

X_train size: 10686, y_train size: 10686
X_val size: 1336, y_val size: 1336
X_test size: 1336, y_test size: 1336


#Model 1 TF-IDF

Build a model to train a tf-idf text classifier. Use a simple logistic regression model for the classifier.

For this part, you may find this [tutorial](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html#sphx-glr-auto-examples-text-plot-document-classification-20newsgroups-py) helpful.

Below are some design choices you need to consider to accomplish this task. Be sure to answer them when you submit your model.

What tokenizer will you use? Why?

**Ans:** I use PyThaiNLP's word_tokenize() with the newmm engine because it is a widely used, rule-based tokenizer for the Thai language.

Will you ignore some stop words (a, an, the, to, etc. for English) in your tf-idf? Is it important?
PythaiNLP provides a list of stopwords if you want to use (https://pythainlp.org/docs/2.0/api/corpus.html#pythainlp.corpus.common.thai_stopwords)

**Ans:** Yes, I remove stopwords using PyThaiNLP's thai_stopwords(). It's important because it help improving the model's ability to focus on meaningful words

The dictionary of TF-IDF is usually based on the training data. How many words in the test set are OOVs?

**Ans:** 415


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_stopwords

In [ ]:
import time

# Tokenizer function
def custom_tokenizer(text):
    return word_tokenize(text, engine="newmm")  # Use Thai word segmentation

# Define stopwords
thai_stop_words = list(thai_stopwords())

# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(
    tokenizer=custom_tokenizer,
    stop_words=thai_stop_words,
    max_features=5000
)

model = Pipeline([
    ("tfidf", tfidf_vectorizer),
    ("clf", LogisticRegression(random_state=42))
])

start_time = time.time()
model.fit(X_train, y_train)

print("Time used: ", time.time() - start_time)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['กระไร', 'กาลนาน', 'ชิ้น', 'ดังที่', 'ดี', 'ดีกว่า', 'ด้อย', 'ตัว', 'ต่อไป', 'ถัดไป', 'ทั่วถึง', 'ทำ', 'ที่จะ', 'ท่าน', 'ท้าย', 'นา', 'บอ', 'บัด', 'ระยะเวลา', 'ล่ะ', 'วันวาน', 'สม', 'สมบูรณ์', 'สํา', 'หน้า', 'หรับ', 'หา', 'อย', 'เกี่ยว', 'เก่า', 'เดี๋ยวนี้', 'เย็น', 'เล่า', 'เสมือน', 'เหมือนกัน', 'แด่', 'แม้น', 'แหล่', 'โง้น', 'โน้น', 'ใด', 'ไว', 'ไหม', '\ufeff'] not in stop_words.
  warnings.warn(


Time used:  5.657778263092041


In [ ]:
# Evaluate Model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Find Out-of-Vocabulary (OOV) Words
train_vocab = set(tfidf_vectorizer.get_feature_names_out())
test_vocab = set(word for text in X_test for word in custom_tokenizer(text))
oov_words = test_vocab - train_vocab

print(f"Number of OOV words: {len(oov_words)}")

              precision    recall  f1-score   support

           0       0.60      0.77      0.67        64
           1       0.58      0.72      0.64       179
           2       0.64      0.64      0.64        73
           3       0.63      0.83      0.72       178
           4       0.60      0.52      0.56        58
           5       0.65      0.70      0.68       210
           6       1.00      0.42      0.59        24
           7       0.76      0.76      0.76       148
           8       0.75      0.27      0.40        33
           9       0.88      0.69      0.77        54
          10       0.93      0.78      0.85        18
          11       0.61      0.55      0.58       115
          12       0.62      0.29      0.39        28
          13       0.00      0.00      0.00         2
          14       0.90      0.76      0.83        25
          15       0.83      0.80      0.82        25
          16       0.67      0.28      0.39        29
          17       0.90    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Measure the time taken for inference
start_time = time.time()

y_pred = model.predict(X_test)

end_time = time.time()

# Calculate elapsed time
print(f"Inference time: {end_time - start_time:.2f} seconds")

Inference time: 0.48 seconds


# Model 2 MUSE

Build a simple logistic regression model using features from the MUSE model.

Which MUSE model will you use? Why?

**Ans:**  sentence-transformers/use-cmlm-multilingual from Hugging Face. beacause, It is optimized for multilingual text embedding, including Thai.


MUSE is typically used with tensorflow. However, there are some pytorch conversions made by some people.

https://huggingface.co/sentence-transformers/use-cmlm-multilingual
https://huggingface.co/dayyass/universal-sentence-encoder-multilingual-large-3-pytorch

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
start_time = time.time()

muse_model = SentenceTransformer("sentence-transformers/use-cmlm-multilingual")

# Convert text into embeddings
X_train_embed = muse_model.encode(X_train.tolist())
X_val_embed = muse_model.encode(X_val.tolist())
X_test_embed = muse_model.encode(X_test.tolist())

clf = LogisticRegression(solver='lbfgs', multi_class='auto')
clf.fit(X_train_embed, y_train)

print("Time used: ", time.time() - start_time)

Some weights of the model checkpoint at sentence-transformers/use-cmlm-multilingual were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1

Time used:  40.63351273536682


In [ ]:
# Evaluate on Test Set
y_test_pred = clf.predict(X_test_embed)
print("Test Set Performance:\n", classification_report(y_test, y_test_pred))

Test Set Performance:
               precision    recall  f1-score   support

           0       0.68      0.75      0.71        64
           1       0.63      0.73      0.68       179
           2       0.67      0.74      0.70        73
           3       0.62      0.80      0.70       178
           4       0.61      0.52      0.56        58
           5       0.74      0.80      0.77       210
           6       0.73      0.33      0.46        24
           7       0.72      0.80      0.75       148
           8       0.79      0.33      0.47        33
           9       0.74      0.57      0.65        54
          10       1.00      0.61      0.76        18
          11       0.67      0.61      0.64       115
          12       0.53      0.29      0.37        28
          13       0.00      0.00      0.00         2
          14       0.90      0.72      0.80        25
          15       0.90      0.76      0.83        25
          16       0.58      0.52      0.55        29
    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Measure the time taken for inference
start_time = time.time()

X_test_embed = muse_model.encode(X_test.tolist())
y_test_pred = clf.predict(X_test_embed)

end_time = time.time()

# Calculate elapsed time
print(f"Inference time: {end_time - start_time:.2f} seconds")

Inference time: 2.67 seconds


# Model 3 WangchanBERTa

We ask you to train a WangchanBERTa-based model.

We recommend you use the thaixtransformers fork (which we used in the PoS homework).
https://github.com/PyThaiNLP/thaixtransformers

The structure of the code will be very similar to the PoS homework. You will also find the huggingface [tutorial](https://huggingface.co/docs/transformers/en/tasks/sequence_classification) useful. Or you can also add a softmax layer by yourself just like in the previous homework.

Which WangchanBERTa model will you use? Why? (Don't forget to clean your text accordingly).

**Ans:** airesearch/wangchanberta-base-att-spm-uncased


In [ ]:
pip install --upgrade protobuf transformers sentencepiece


In [ ]:
pip install -U datasets


In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate

In [ ]:
import time
import numpy as np
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
)
from datasets import Dataset
import evaluate

In [ ]:
import time
start_time = time.time()

model_name = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=416)

X_train_tokenized = tokenizer(X_train.tolist(), truncation=True, padding=True)
X_val_tokenized = tokenizer(X_val.tolist(), truncation=True, padding=True)
X_test_tokenized = tokenizer(X_test.tolist(), truncation=True, padding=True)

train_dataset = Dataset.from_dict({
    "input_ids": X_train_tokenized["input_ids"],
    "attention_mask": X_train_tokenized["attention_mask"],
    "labels": y_train
})

val_dataset = Dataset.from_dict({
    "input_ids": X_val_tokenized["input_ids"],
    "attention_mask": X_val_tokenized["attention_mask"],
    "labels": y_val
})

test_dataset = Dataset.from_dict({
    "input_ids": X_test_tokenized["input_ids"],
    "attention_mask": X_test_tokenized["attention_mask"],
    "labels": y_test
})

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(label_2_idx), id2label=idx_2_label, label2id=label_2_idx
)

training_args = TrainingArguments(
    output_dir="text_classification_wangchanberta_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train model
trainer.train()

print("Time used: ", time.time() - start_time)

model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-30-a889cc23125e>:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `Training

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kknot2004 (kknot2004-chulalongkorn-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,2.475700,1.450655,0.535928
2,1.537200,1.048874,0.684132
3,0.807200,0.830850,0.755240
4,0.627100,0.816257,0.752994
5,0.565600,0.818303,0.753743


Time used:  1450.3295040130615


In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.8162572383880615, 'eval_accuracy': 0.7529940119760479, 'eval_runtime': 5.4047, 'eval_samples_per_second': 247.193, 'eval_steps_per_second': 15.542, 'epoch': 5.0}


In [ ]:
import torch
# Measure the time taken for inference
start_time = time.time()

X_test_tokenized = tokenizer(X_test.tolist(), truncation=True, padding=True, return_tensors="pt")

# Run inference
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
X_test_tokenized = {key: val.to(device) for key, val in X_test_tokenized.items()}
with torch.no_grad():
    outputs = model(**X_test_tokenized)

end_time = time.time()

# Calculate elapsed time
print(f"Inference time: {end_time - start_time:.2f} seconds")

Inference time: 0.52 seconds


After you

# Comparison

After you have completed the 3 models, compare the accuracy, ease of implementation, and inference speed (from cleaning, tokenization, till model compute) between the three models in mycourseville.